Thymio mini project

In [1]:
!pip install --upgrade tdmclient

In [50]:
# import libraries
import time
from tdmclient import ClientAsync
import numpy as np
import math

# import project files
import constants as cst
import extended_Kalman_filter as eKf
import vision
import geometry as geo


In [51]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node df054775-a7a5-454e-9ad3-7896a54bbcc8

In [52]:
# Set the motor speeds
async def stop_robot():
    """
    Stop the robot. Set the motor speed of each wheel to zero.
    """
    await node.set_variables({
            "motor.left.target": [0],
            "motor.right.target": [0],
        })

async def move_robot(left_speed, right_speed):
    """
    Turn the robot wheels to the given speeds.
    Parameter: - left_speed    : left wheel speed in [thymio speed]
               - right_speed   : right wheel speed in [thymio speed]
    """
    await node.set_variables({
            "motor.left.target": [left_speed],
            "motor.right.target": [right_speed],
        })

In [53]:
# Set the sensor values
async def get_sensor_values():
    """
    Get the sensor measurements of a given sensor.
    Parameter: - sensor_id    : name of the sensor (a string)
    """
    
    await node.wait_for_variables({str('prox.horizontal')})
    prox_sensors = np.asarray(list(node['prox.horizontal']))
    prox_sensors = prox_sensors[0:5]

    prox_sensors_bool = (prox_sensors >= np.ones(5) * 50) #cst.THRESHOLD_PROX_SENSOR
    detected = False
    if(any(prox_sensors_bool)):
        detected = True

    idx = np.argmax(prox_sensors)

    return prox_sensors, detected, idx

In [54]:
# test thymio
# move the robot forward and get prox values.
await move_robot(50, 50)
dt = 0
old_time = time.time()
while(dt < 2):
    y,d,i = await get_sensor_values()
    print(y)
    print(d)
    print(i)
    await client.sleep(0.1)
    dt = time.time() - old_time
await stop_robot()

[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0
[0 0 0 0 0]
False
0


In [57]:

# state variables
global_state = 'rotation'

### initialisation
# vision (détection robot, obstacles, goal)
#obstacles, goal_centroid, robot_centroid, robot_direction = vision.vision_obstacles_positions()
# global nav

intermediate_goals = [(30, 0)]
robot = [0.0,0.0]
robot_orientation = 0
robot_speed = [0.0,0.0]

Kfilter = eKf.Kalman(robot[0], robot[0], robot_orientation)


old_time = 0.0

iter = 0
while(True):
    print(iter)
    if intermediate_goals == []:
        await stop_robot()
        break

    # vision et gobale toutes les n iterations (i modulo n == 0)
    
    # get robot sensor values
    #prox_sensors, detected, max_index = await get_sensor_values()
    # test if local (with proximity)
    #if(detected):
    #    global_state = 'local_avoidance'
    
    if iter == 0:                   # first time
        old_time = time.time()
    dt = time.time() - old_time     # get time difference for Kalman
    print("dt: ", dt)
    # Kalman (with wheel speed and camera position)
    old_time = time.time()

    #robot[0] = robot[0] + dt * ((robot_speed[0] + robot_speed[1])/(2.0*18.1818)) * math.cos(robot_orientation)
    #robot[1] = robot[1] + dt * ((robot_speed[0] + robot_speed[1])/(2.0*18.1818)) * math.sin(robot_orientation)
    #robot_orientation = robot_orientation + dt * (robot_speed[0] - robot_speed[1])/(4.0*cst.WHEELS_DIST)
    #print(robot[0]," ", robot[1]," ", robot_orientation)
    
    Kfilter.dt_update(dt)
    Kfilter.measurement_wheels(robot_speed[0] / 18.1818, robot_speed[1] / 18.1818)
    Kfilter.Kalman_filter()
    print("Kalman: ", Kfilter.Mu)

    if global_state == 'rotation':
        angle = Kfilter.Mu[2] - np.arctan2(intermediate_goals[0][1] - Kfilter.Mu[1], intermediate_goals[0][0] - Kfilter.Mu[0])
        #angle = robot_orientation - np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0])
        print("rotation"
        )#print("rotation :", robot_orientation, np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0]))
        
        if abs(angle) < 0.1:
            global_state = 'translation'
            continue

        # call rotation fct
        if(angle > 0):
            robot_speed = [-20, 20]
        else:
            robot_speed = [20, -20]
        await move_robot(robot_speed[0], robot_speed[1])
        

    elif global_state == 'translation':
        print("translation")
        #print("translation: ", robot[0], " ", robot[1], "  ", math.dist(intermediate_goals[0], robot))
        # if goal reached
            # exit
        #if abs(intermediate_goals[0][0] - Kfilter.Mu[0]) < 1 and abs(intermediate_goals[0][1] - Kfilter.Mu[1]) < 1:
        if abs((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2) < 1:
        #if abs((intermediate_goals[0][0] - robot[0])**2 - (intermediate_goals[0][1] - robot[1])**2) < 2:
            intermediate_goals.pop()[0]
            global_state = 'rotation'
            continue
        
        # call translation fct
        robot_speed = [50, 50]
        await move_robot(robot_speed[0], robot_speed[1])

    elif global_state == 'local_avoidance':
        # call local_avoidance fct (= add obstacle on global map and recompute global nav)
        
        global_state = 'rotation'
    time.sleep(0.05)

    iter = iter + 1

0
dt:  0.0
Kalman:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
rotation
0
dt:  0.0
Kalman:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
translation
1
dt:  0.23475980758666992
Kalman:  [[1.48982886e-01]
 [0.00000000e+00]
 [1.24520177e-20]
 [1.69231056e+00]
 [1.69231056e+00]]
translation
2
dt:  0.15406370162963867
Kalman:  [[4.69957490e-01]
 [2.14622819e-21]
 [5.36807645e-20]
 [2.34559147e+00]
 [2.34559147e+00]]
translation
3
dt:  0.15511012077331543
Kalman:  [[8.57384259e-01]
 [2.10749438e-20]
 [3.70972611e-21]
 [2.59550872e+00]
 [2.59550872e+00]]
translation
4
dt:  0.1581571102142334
Kalman:  [[ 1.27709423e+00]
 [ 2.25481175e-20]
 [-8.54539060e-21]
 [ 2.69099007e+00]
 [ 2.69099007e+00]]
translation
5
dt:  0.15849065780639648
Kalman:  [[1.70714295e+00]
 [1.89133970e-20]
 [5.25556251e-20]
 [2.72746187e+00]
 [2.72746187e+00]]
translation
6
dt:  0.16108918190002441
Kalman:  [[ 2.14788296e+00]
 [ 4.19835422e-20]
 [-1.10127533e-20]
 [ 2.74139290e+00]
 [ 2.74139290e+00]]
translation
7
dt:  0.15702366828918457
Kalman:

In [56]:
await stop_robot()